In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import _pickle as cPickle

In [2]:
def unpickle(filename):
    fo = open(filename, 'rb')
    fdict = cPickle.load(fo, encoding='latin1')
    fo.close()
    return fdict

In [3]:
data_dict = {}
for i in range(1, 6):
    data_dict['data_batch_' + str(i)] = unpickle('cucumber_data/p1/data_batch_' + str(i))
data_dict['test_batch'] = unpickle('cucumber_data/p1/test_batch')

In [4]:
def get_images(data, labels):
    num_images = data.shape[0] // 3
    images = np.zeros((num_images, 32*32*3))
    labels_arr = np.zeros((num_images, 9))
    for i in range(num_images):
        images[i] = np.hstack((data[3*i], data[3*i+1], data[3*i+2])) / 255
        labels_arr[i][labels[i]] = 1
    return (images, labels_arr)

In [5]:
images_dict = {}
labels_dict = {}
for data_batch in data_dict:
    images_dict[data_batch], labels_dict[data_batch] = get_images(data_dict[data_batch]['data'], data_dict[data_batch]['labels'])

In [6]:
validation_images = images_dict['data_batch_5']
validation_labels = labels_dict['data_batch_5']

test_images = images_dict['test_batch']
test_labels = labels_dict['test_batch']

train_images = np.vstack((images_dict['data_batch_1'], 
                          images_dict['data_batch_2'], 
                          images_dict['data_batch_3'], 
                          images_dict['data_batch_4']))

train_labels = np.vstack((labels_dict['data_batch_1'], 
                          labels_dict['data_batch_2'], 
                          labels_dict['data_batch_3'], 
                          labels_dict['data_batch_4']))

In [7]:
train_images.shape

(1980, 3072)

In [8]:
train_labels.shape

(1980, 9)

In [9]:
def get_batch(images, batch_size):
    images_ind = list(range(len(images)))
    random.shuffle(images_ind)
    return images[images_ind[:batch_size]]

## Deep Neural Network

In [10]:
input_size = 32*32*3
neuron_1 = 300
neuron_2 = 300
# neuron_3 = 100
# neuron_4 = 50

learning_rate = 1e-4
num_train_examples = train_images.shape[0]
iters = 10*num_train_examples
batch_size = 100
num_subjects = 9
keep_prob = tf.placeholder(tf.float32)

In [11]:
x = tf.placeholder(shape=[None, input_size], dtype=tf.float32)
y_correct = tf.placeholder(shape=[None, num_subjects], dtype=tf.float32)

In [12]:
W_1 = tf.Variable(tf.truncated_normal(shape=[input_size, neuron_1], stddev=0.1))
b_1 = tf.Variable(tf.truncated_normal(shape=[neuron_1], stddev=0.1))

z_1 = tf.add(tf.matmul(x, W_1), b_1)
a_1 = tf.nn.dropout(tf.nn.relu(z_1), keep_prob=keep_prob)

In [13]:
W_2 = tf.Variable(tf.truncated_normal(shape=[neuron_1, neuron_2], stddev=0.1))
b_2 = tf.Variable(tf.truncated_normal(shape=[neuron_2], stddev=0.1))

z_2 = tf.add(tf.matmul(a_1, W_2), b_2)
a_2 = tf.nn.dropout(tf.nn.relu(z_2), keep_prob=keep_prob)

In [14]:
##### 3 layers
#W_3 = tf.Variable(tf.truncated_normal(shape=[neuron_2, neuron_3], stddev=0.1))
#b_3 = tf.Variable(tf.truncated_normal(shape=[neuron_3], stddev=0.1))
#z_3 = tf.add(tf.matmul(a_2, W_3), b_3)
#a_3 = tf.nn.dropout(tf.nn.relu(z_3), keep_prob=keep_prob)
#W_4 = tf.Variable(tf.truncated_normal(shape=[neuron_3, num_subjects], stddev=0.1))
#b_4 = tf.Variable(tf.truncated_normal(shape=[num_subjects], stddev=0.1))
#y = tf.add(tf.matmul(a_3, W_4), b_4)
##### End of 3 layers

In [15]:
##### 2 layers
W_3 = tf.Variable(tf.truncated_normal(shape=[neuron_2, num_subjects], stddev=0.1))
b_3 = tf.Variable(tf.truncated_normal(shape=[num_subjects], stddev=0.1))
y = tf.add(tf.matmul(a_2, W_3), b_3)
##### End of 2 layers

In [16]:
##### 4 layers
# W_3 = tf.Variable(tf.truncated_normal(shape=[neuron_2, neuron_3], stddev=0.1))
# b_3 = tf.Variable(tf.truncated_normal(shape=[neuron_3], stddev=0.1))
# z_3 = tf.add(tf.matmul(a_2, W_3), b_3)
# a_3 = tf.nn.relu(z_3)
# W_4 = tf.Variable(tf.truncated_normal(shape=[neuron_3, neuron_4], stddev=0.1))
# b_4 = tf.Variable(tf.truncated_normal(shape=[neuron_4], stddev=0.1))
# z_4 = tf.add(tf.matmul(a_3, W_4), b_4)
# a_4 = tf.nn.relu(z_4)
# W_5 = tf.Variable(tf.truncated_normal(shape=[neuron_4, num_subjects], stddev=0.1))
# b_5 = tf.Variable(tf.truncated_normal(shape=[num_subjects], stddev=0.1))
# y = tf.add(tf.matmul(a_4, W_5), b_5)
##### End of 4 layers

In [17]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels = y_correct))

train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_correct,1), name = "correct_prediction")
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

validation_feed_dict = {x: validation_images, y_correct: validation_labels, keep_prob:1.0 }
test_feed_dict = {x: test_images, y_correct: test_labels, keep_prob:1.0 }

In [18]:
for i in range(iters):
    train_indices = random.sample(range(num_train_examples), batch_size)
    train_feed_dict = {x: train_images[train_indices], y_correct: train_labels[train_indices], keep_prob:0.7 }
    if i%2000 == 0:
        # print accuracy
        train_acc = sess.run(accuracy, feed_dict=train_feed_dict)
        validation_acc = sess.run(accuracy, feed_dict=validation_feed_dict)
        print("STEP {}".format(i))
        print("Train accuracy: {}".format(train_acc))
        print("Validation accuracy: {}".format(validation_acc))

    _ = sess.run(train_step, feed_dict = train_feed_dict)

test_acc = sess.run(accuracy, feed_dict=test_feed_dict)
print("DONE. Test accuracy: {}".format(test_acc))

STEP 0
Train accuracy: 0.12999999523162842
Validation accuracy: 0.111111119389534
STEP 2000
Train accuracy: 0.6299999356269836
Validation accuracy: 0.6727272868156433
STEP 4000
Train accuracy: 0.7599999904632568
Validation accuracy: 0.7191920280456543
STEP 6000
Train accuracy: 0.8199999332427979
Validation accuracy: 0.7494949698448181
STEP 8000
Train accuracy: 0.8399999737739563
Validation accuracy: 0.765656590461731
STEP 10000
Train accuracy: 0.9200000166893005
Validation accuracy: 0.7838383913040161
STEP 12000
Train accuracy: 0.9200000762939453
Validation accuracy: 0.7737374305725098
STEP 14000
Train accuracy: 0.9099999666213989
Validation accuracy: 0.7757576704025269
STEP 16000
Train accuracy: 0.9600000381469727
Validation accuracy: 0.7838384509086609
STEP 18000
Train accuracy: 0.9300000071525574
Validation accuracy: 0.7757575511932373
DONE. Test accuracy: 0.802020251750946
